In [1]:
import os
import backtrader as bt
import pandas as pd
import argparse


def runstrat(args=None):
    args = parse_args(args)

    cerebro = bt.Cerebro()
    cerebro.broker.setcash(args.cash)

    dkwargs = dict()
    if args.fromdate is not None:
        dt_start = bt.datetime.strptime(args.fromdate, '%Y-%m-%d')
    else:
        dt_start = bt.datetime.datetime.strptime('20220101', "%Y%m%d")
    if args.todate is not None:
        dt_end = bt.datetime.strptime(args.todate, '%Y-%m-%d')
    else:
        dt_end = bt.datetime.datetime.strptime('20220222', "%Y%m%d")

    dkwargs['fromdate'] = dt_start
    dkwargs['todate'] = dt_end

    myQuant_ROOT = os.getcwd()[:os.getcwd().find("bt_backtrader\\") + len("bt_backtrader\\")]  # 获取项目中相对根路径
    file_path = os.path.join(myQuant_ROOT, args.data)  #文件路径

    if not os.path.exists(file_path):
        print("数据源文件未找到！" + file_path)
        raise Exception("数据源文件未找到！" + file_path)

    # 从pandas加载数据
    data = bt.feeds.BacktraderCSVData(dataname=file_path, **dkwargs)
    cerebro.adddata(data)

    # strategy
    cerebro.addstrategy(DemoStrategy)

    cerebro.run()

    if args.plot:
        pkwargs = dict(style='bar')
        if args.plot is not True:  # evals to True but is not True
            npkwargs = eval('dict(' + args.plot + ')')  # args were passed
            pkwargs.update(npkwargs)

        cerebro.plot(**pkwargs)


def parse_args(pargs=None):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Sample for Order Target')

    parser.add_argument('--data', required=False,
                        default='datas\\SQRB13-5m-20180702-20220319.csv',
                        help='Specific data to be read in')

    parser.add_argument('--fromdate', required=False, default=None,
                        help='Starting date in YYYY-MM-DD format')

    parser.add_argument('--todate', required=False, default=None,
                        help='Ending date in YYYY-MM-DD format')
    # Plot options
    parser.add_argument('--plot', '-p', nargs='?', required=False,
                        metavar='kwargs', const=True,
                        help=('Plot the read data applying any kwargs passed\n'
                              '\n'
                              'For example:\n'
                              '\n'
                              '  --plot style="candle" (to plot candles)\n'))
    if pargs is not None:
        return parser.parse_args(pargs)

    return parser.parse_args()

In [2]:

# 演示用策略，每日输出开盘价
class DemoStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        """ Logging function for this strategy"""
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    params = dict(
        RSPP=10,  # 盈利千分比
        OCJK=0,  # CLOSE与OPEN的间隔
        POSMIN=10,  # 最小开仓单位 按(数量,金额,百分比)下单
        POSMAX=0,  # 最大开仓单位
        SSPP=0,  # 最大回撤千分比
        addLongOrShort=0,  #加仓方向addLongOrShort=0无限制,>0时只有多头加仓,<0时只有空头加仓
        valid=None,  # 订单生效时间
        use_target_size=False,  # 按目标数量下单
        use_target_value=False,  # 按目标金额下单
        use_target_percent=True,  # 按目标百分比下单
    )

    def __init__(self):
        print("【策略】init 函数 开始")
        self.orderid = list()
        self.order = None

        self.mprs = self.p.RSPP / 1000  # 盈亏升分比

        self.buyorderthisbar = 0  # 该周期是否有交易 0没有,1有

        self.myentryprice = 0.0  # 入场价格
        self.myexitprice = 0.0  # 离场价格
        self.turtleunits = 0  # 加仓次数

        self.numlosst = 0  # 统计连续亏损次数
        self.ppunit = 1  # 交易手数比率
        self.positionflag = 0  # 仓位状态 0表示没有仓位，1表示持有多头， -1表示持有空头

        self.radd = 0  # 盈利加仓价格
        self.sexit = 0  # 亏损退出价格
        self.sigLong = 0  # 多头条件
        self.sigShort = 0  # 空头条件
        self.longa1 = 0  # 多头加仓条件
        self.shorta1 = 0  # 空头条件
        self.longx1 = 0  # 多头离场条件
        self.shortx1 = 0  # 空头离场条件
        self.mbstop = 0  # 是否触发回撤止损
        self.pl_sum = 0  # 累计连续亏损百分比

        # 建立对于DataFeed的Open/Close价格的引用参数
        self.dtopen = self.datas[0].open
        self.dtclose = self.datas[0].close
        self.dthigh = self.datas[0].high
        self.dtlow = self.datas[0].low
        self.jOpen = self.jOpen if abs(self.dtopen - self.jOpen) < self.p.OCJK else self.dtopen
        # 多头入场条件
        self.sigLong = ((self.dtclose > self.jOpen)
                        and abs(self.dtclose - self.jOpen) > self.p.OCJK
                        and not self.position)
        # 空头入场条件
        self.sigShort = ((self.dtclose < self.jOpen)
                         and abs(self.dtclose - self.jOpen) > self.p.OCJK
                         and not self.position)

        # 多头加仓条件
        self.longa1 = (self.positionflag == 1) and (self.dthigh > self.dtlow) and (
                self.dtclose >= self.radd) and (self.buyorderthisbar == 0)
        # 空头加仓条件
        self.shorta1 = (
                self.positionflag == -1 and self.dthigh > self.dtlow and (
                self.dtclose <= self.radd) and self.buyorderthisbar == 0)

        # 空头离场条件 添加OPEN价离场条件
        self.shortx1 = (self.positionflag == -1) and self.dthigh > self.dtlow and (
                self.dtclose >= self.sexit or self.dtopen >= self.sexit)
        # 多头离场条件 添加OPEN价离场条件
        self.longx1 = (self.positionflag == 1) and self.dthigh > self.dtlow and (
                self.dtclose <= self.sexit or self.dtopen <= self.sexit)
        print("【策略】init 函数 结束")

    # 整个回测周期上，不同时间段对应的函数
    def start(self):
        """在回测开始之前调用,对应第0根bar"""
        # 回测开始之前的有关处理逻辑可以写在这里
        # 默认调用空的 start() 函数，用于启动回测
        pass

    def prenext(self):
        """策略准备阶段,对应第1根bar ~ 第 min_period-1 根bar"""
        # 该函数主要用于等待指标计算，指标计算完成前都会默认调用prenext()空函数
        # min_period 就是 __init__ 中计算完成所有指标的第1个值所需的最小时间段
        pass

    def nextstart(self):
        """策略正常运行的第一个时点，对应第 min_period 根bar"""
        # 只有在 __init__ 中所有指标都有值可用的情况下，才会开始运行策略
        # nextstart()只运行一次，主要用于告知后面可以开始启动 next() 了
        # nextstart()的默认实现是简单地调用next(),所以next中的策略逻辑从第 min_period根bar就已经开始执行
        pass

    def next(self):
        """策略正常运行阶段，对应第min_period+1根bar ~ 最后一根bar"""
        # 主要的策略逻辑都是写在该函数下
        # 进入该阶段后，会依次在每个bar上循环运行next函数
        self.buyorderthisbar = 0  # 该周期是否有交易 0没有,1有
        dt = self.data.datetime.date()
        if len(self) < 3:
            # 查询函数
            print('当前持仓量', self.getposition(self.data).size)
            print('当前持仓成本', self.getposition(self.data).price)
            print('头寸信息', self.getpositionbyname(name=None, broker=None))
            print('数据集名称列表', self.getdatanames())
        if self.orderId:
            return  # if an order is active, no new orders are allowed

        size = self.p.POSMIN
        # 按目标数量下单
        if self.p.use_target_size:
            print('%04d - %s - Order Target Size: %02d' %
                  (len(self), dt.isoformat(), size))
            self.order = self.order_target_size(target=size)
        # 按目标金额下单
        elif self.p.use_target_value:
            value = size * 1000
            print('%04d - %s - Order Target Value: %.2f' %
                  (len(self), dt.isoformat(), value))
            self.order = self.order_target_value(target=value)
        # 按目标百分比下单
        elif self.p.use_target_percent:
            percent = size / 100.0
            print('%04d - %s - Order Target Percent: %.2f' %
                  (len(self), dt.isoformat(), percent))
            self.order = self.order_target_percent(target=percent)
        # 没有持仓时
        if not self.position and len(self.orderid) < 1:

            if self.sigLong or self.sigShort:
                self.myentryprice = self.jOpen
                self.turtleunits = 1
                self.buyorderthisbar = 1

            if self.sigLong:
                self.positionflag = 1
                # self.log('CLOSE SHORT , %.2f' % self.data.close[0])
                # self.order = self.close()  # 平仓 cover
                self.log('BUY CREATE , %.2f' % self.data.close[0])
                self.order = self.buy(size=size)  # 买入、做多 long
                self.radd = self.myentryprice * (1 + self.mprs)  # 多头盈利加仓目标
                self.sexit = self.myentryprice / (1 + self.mprs)  # 多头亏损退出目标
                self.order = self.buy(exectype=bt.Order.Limit,  # 限制开仓订单
                                      price=self.myentryprice,
                                      valid=self.p.valid,  # 订单生效时间
                                      transmit=False)  # 默认True 指示是否必须传输订单，即：不仅放置在经纪人中，而且还发出。

                self.sell(exectype=bt.Order.StopTrail,  # 追踪止损订单
                          price=self.sexit,
                          valid=self.p.valid,  # 订单生效时间
                          parent=self.order,
                          transmit=False)

            elif self.sigShort:
                self.positionflag = -1
                # self.log('CLOSE LONG , %.2f' % self.data.close[0])
                # self.order = self.close()  # 平仓 cover
                self.log('SELL CREATE , %.2f' % self.data.close[0])
                self.order = self.sell(size=size)  # 卖出、做空 short
                self.radd = self.myentryprice / (1 + self.mprs)  # 空头盈利加仓目标
                self.sexit = self.myentryprice * (1 + self.mprs)  # 空头亏损退出目标

                self.order = self.sell(exectype=bt.Order.Limit,  # 限制开仓订单
                                       price=self.myentryprice,
                                       valid=self.p.valid,  # 订单生效时间
                                       transmit=False)  # 默认True 指示是否必须传输订单，即：不仅放置在经纪人中，而且还发出。

                self.sell(exectype=bt.Order.StopTrail,  # 追踪止损订单
                          price=self.sexit,
                          valid=self.p.valid,  # 订单生效时间
                          parent=self.order,
                          transmit=False)

            self.orderid.append(self.order)
        elif (self.longa1 or self.shorta1) or (self.shortx1 or self.longx1):  # 持有头寸时,满足加仓或减仓条件时

            # 加仓共有条件
            if self.longa1 or self.shorta1:
                self.myentryprice = self.radd
                self.turtleunits = self.turtleunits + 1

                self.buyorderthisbar = 1

            pass


# data = self.getdatabyname('data0')  # 根据名称返回数据集

# # 订单组合
# brackets = self.buy_bracket()
# brackets = self.sell_bracket()
pass


def stop(self):
    """策略结束，对应最后一根bar"""
    # 告知系统回测已完成，可以进行策略重置和回测结果整理了
    pass


# 打印回测日志
def notify_order(self, order):
    """通知订单信息"""
    pass


def notify_trade(self, trade):
    """通知交易信息"""
    pass


def notify_cashvalue(self, cash, value):
    """通知当前资金和总资产"""
    pass


def notify_fund(self, cash, value, fundvalue, shares):
    """返回当前资金、总资产、基金价值、基金份额"""
    pass


def notify_store(self, msg, *args, **kwargs):
    """返回供应商发出的信息通知"""
    pass


def notify_data(self, data, status, *args, **kwargs):
    """返回数据相关的通知"""
    pass


def notify_timer(self, timer, when, *args, **kwargs):
    """返回定时器的通知"""
    # 定时器可以通过函数add_time()添加
    pass


# 启动回测
def engine_run():
    # 初始化引擎
    cerebro = bt.Cerebro()

    # 给Cebro引擎添加策略
    cerebro.addstrategy(DemoStrategy)

    # 设置初始资金：
    cerebro.broker.setcash(100000.0)

    # 设置交易费用
    cerebro.broker.setcommission(
        commission=0.1,  # 0.1% # 交易手续费，根据margin取值情况区分是百分比手续费还是固定手续费
        margin=2000,  # 期货保证金，决定着交易费用的类型,只有在stocklike=False时起作用
        mult=10,  # 乘数，盈亏会按该乘数进行放大
        stocklike=False,  # 指定为期货模式
        percabs=False,  # commission 以 % 为单位
        # 交易费用计算方式，取值有：
        # 1.CommInfoBase.COMM_PERC 百分比费用
        # 2.CommInfoBase.COMM_FIXED 固定费用
        # 3.None 根据 margin 取值来确定类型
        commtype=bt.CommInfoBase.COMM_PERC,  # 使用百分比费用模式
    )
    # 日期格式转换

    dt_start = bt.datetime.datetime.strptime('20220101', "%Y%m%d")
    dt_end = bt.datetime.datetime.strptime('20220222', "%Y%m%d")
    # 从pandas加载数据
    # data = bt.feeds.PandasData(dataname=df, fromdate=dt_start, todate=dt_end)
    data = bt.feeds.PandasData(dataname=df)
    cerebro.adddata(data)

    print('初始市值: %.2f' % cerebro.broker.getvalue())
    # 回测启动运行

    result = cerebro.run()
    print("回测运行返回值 = {0}".format(result))
    print('期末市值: %.2f' % cerebro.broker.getvalue())


if __name__ == '__main__':
    engine_run()

    # os.system("pause")

NameError: name 'df' is not defined

In [ ]:

print(1 if not False else 3)